# ML專案:Boston  house prices prediction

https://www.kaggle.com/sagarnildass/predicting-boston-house-prices

https://github.com/antra0497/Udacity-Predicting-Boston-Housing-Prices/blob/master/boston_housing.ipynb

https://github.com/DavidCico/Boston-House-Prices-With-Regression-Machine-Learning-and-Keras-Deep-Learning/blob/master/Boston_deeplearning_regression.ipynb

波士頓房價( Boston House Price) 資料集
```
資料集中的每一行資料都是對波士頓周邊或城鎮房價的描述。
資料是1978 年統計收集的，包含以下14 個特徵和506 條資料。
```
```
CRIM: 城鎮人均犯罪率。
ZN: 住宅用地所占比例。
lNDUS: 城鎮中非住宅用地所占比例。
CHAS: 虛擬變數，用於回歸分析。
NOX: 環保指數。
RM: 每棟住宅的房間數。
AGE: 1940 年以前建成的自住單位的比例。
DIS: 距離5 個波士頓的就業中心的加權距離。
RAD: 距離高速公路的便利指數。
TAX: 每一萬美元的不動產稅率。
PTRATIO: 城鎮中教師和學生的比例。
B: 城鎮中黑人的比例。
LSTAT: 地區中有多少房東屬於低收入人群。
MEDV: 自住房屋房價中位數。
```

In [0]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


Using TensorFlow backend.


### 載入資料集

In [0]:
dataset = datasets.load_boston()

x = dataset.data
Y = dataset.target



In [0]:
x[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [0]:
# 設定隨機種子
seed = 7
np.random.seed(seed)


In [0]:
# 構建模型函數
def create_model(units_list=[13],optimizer='adam', init='normal'):
    # 構建模型
    model = Sequential()

    # 構建第一個隱藏層和輸入層
    units = units_list[0]
    model.add(Dense(units=units, activation='relu', input_dim=13, kernel_initializer=init))
    # 構建更多隱藏層
    for units in units_list[1:]:
        model.add(Dense(units=units, activation='relu', kernel_initializer=init))

    model.add(Dense(units=1, kernel_initializer=init))

    # 編譯模型
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    return model


In [0]:
model = KerasRegressor(build_fn=create_model, epochs=200, batch_size=5, verbose=0)


# 設置演算法評估基準
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, x, Y, cv=kfold)
print('Baseline: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Baseline: -21.16 (10.52) MSE


##  資料正態化，改進演算法

In [0]:

steps = []
steps.append(('standardize', StandardScaler()))
steps.append(('mlp', model))
pipeline = Pipeline(steps)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, x, Y, cv=kfold)
print('Standardize: %.2f (%.2f) MSE' % (results.mean(), results.std()))


Standardize: -12.65 (6.97) MSE


## 測試不同演算法及其他參數

調整不同參數以便選擇最優模型

In [0]:

param_grid = {}
param_grid['units_list'] = [[20], [13, 6]]
param_grid['optimizer'] = ['rmsprop', 'adam']
param_grid['init'] = ['glorot_uniform', 'normal']
param_grid['epochs'] = [100, 200]
param_grid['batch_size'] = [5, 20]


In [0]:
# 調參
scaler = StandardScaler()
scaler_x = scaler.fit_transform(x)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
results = grid.fit(scaler_x, Y)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [0]:
# 輸出結果
print('Best: %f using %s' % (results.best_score_, results.best_params_))
means = results.cv_results_['mean_test_score']
stds = results.cv_results_['std_test_score']
params = results.cv_results_['params']

for mean, std, param in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, std, param))


Best: -24.553191 using {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop', 'units_list': [13, 6]}
-36.911305 (20.581992) with: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop', 'units_list': [20]}
-24.553191 (16.760687) with: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop', 'units_list': [13, 6]}
-48.017717 (35.475526) with: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam', 'units_list': [20]}
-31.651377 (14.743790) with: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam', 'units_list': [13, 6]}
-57.498927 (40.035703) with: {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer': 'rmsprop', 'units_list': [20]}
-50.858566 (48.022781) with: {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer': 'rmsprop', 'units_list': [13, 6]}
-41.051482 (18.660904) with: {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimizer':